In [ ]:
import optuna
import pandas as pd
import numpy as np
import yaml

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import tensor, cat, save, load, optim, nn
from torch.utils.data import DataLoader
import pickle
from sklearn.model_selection import train_test_split
from models.resnet18_model import ResNet18

ModuleNotFoundError: No module named 'optuna'

In [ ]:
# Optuna objective
def Objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return x**2/(10-x)

In [ ]:
# Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(Objective, n_trials=50)

In [ ]:
print("Best value:", study.best_value)
print("Best params:", study.best_params)

In [ ]:
#-------------------#
#  Prepare dataset  #
#-------------------#
from dataset_handler import processDataset, loadDataset

# Dataset config
data_cfg = { 
            "dataset": "cifar10",
            "f_train": "0.5",
            "f_test": "0.5",
            "root": "./data"
            }

trainset, testset = loadDataset(data_cfg)

# Perpare loaders
train_loader, test_loader, train_indices, test_indices = processDataset(train_cfg, trainset, testset)

ModuleNotFoundError: No module named 'da'

In [ ]:
#-----------------------#
#       Objective       #
#-----------------------#
def objective(trial):
    # Hyperparameters
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    T_max = trial.suggest_int("T_max", 20, 50)

    train_loader, val_loader = get_dataloaders(batch_size, args.augment)

    model = torchvision.models.resnet18(num_classes=10).to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max)

    max_epochs = 50
    for epoch in range(max_epochs):
        train_one_epoch(model, optimizer, train_loader, DEVICE)
        scheduler.step()
        val_accuracy = evaluate(model, val_loader, DEVICE)
        trial.report(val_accuracy, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_accuracy

def train_one_epoch(model, optimizer, train_loader, device):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
# ---------------------------#
#      Run optimization      #
# ---------------------------#
def run_optimization():
    storage = JournalStorage(JournalFileBackend(file_path="./journal.log"))
    study = optuna.create_study(
        study_name=args.study_name,
        storage=storage,
        load_if_exists=True,
        direction="maximize"
    )
    study.optimize(objective, n_trials=args.n_trials)
    print(f"Study '{args.study_name}' completed. Best value: {study.best_value}, params: {study.best_params}")

In [ ]:
# Save training and config metadata. Folder name index-hashed_config/logits, metadata, etc
form save-load import save, hashCfg


In [ ]:
# Train shadow models